## WikiGolf

Goal: To reach a target page in as few links as possible

Description:
WikiGolf is a program intended to browse Wikipedia with some degree of intelligence. It will accept two parameters, a starting page and a target. It will begin at the starting page and check the links for anything related to the target, and if no relations are found it will choose one at random. It will then examine that page and check any of its links for pages related to the target, and so forth until it reaches a page that is within the threshold for relevence to the target. 

In [1]:
import numpy as np
import requests
import wikipedia
import gensim
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import wikisearch
import nltk
from nltk.corpus import stopwords
import nltk.collocations
from nltk import FreqDist, word_tokenize
import string
import re
import sys

### Manual interaction with the Wikipedia API

It is possible to find all links on a page using direct interaction with the Wikipedia API, but accessing the relevent information from the query result is somewhat complex.

In [ ]:
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

TITLE = 'Jurassic Park (novel)'

PARAMS = {
    'action': "query",
    'titles': TITLE,
    'prop': "links",
    'pllimit': "max",
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
data = R.json()
#print(data['query']['title'])

In [ ]:
data['query']['pages'][list(data['query']['pages'])[0]]['links'][:20]

In [ ]:
links = data['query']['pages'][list(data['query']['pages'])[0]]['links']

In [ ]:
n = np.random.randint(0,len(links))
links[n]

In [ ]:
n = np.random.randint(0,len(links))
NEXT = links[n]['title']

PARAMS = {
    'action': "query",
    'titles': NEXT,
    'prop': "links",
    'pllimit': "max",
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
next_data = R.json()
print(links[n]['title'])
print(next_data['query']['pages'][list(next_data['query']['pages'])[0]]['title'])

In [ ]:
def get_links(TITLE):
    PARAMS = {
        'action': "query",
        'titles': TITLE,
        'prop': "links",
        'pllimit': "max",
        'format': "json",
    }

    R = S.get(url=URL, params=PARAMS)
    return R.json()

In [ ]:
start = input()
hops = input()

print("0 : " + start)
title = start
for i in range(int(hops)):
    response = get_links(title)
    links = response['query']['pages'][list(response['query']['pages'])[0]]['links']
    n = np.random.randint(0,len(links))
    title = links[n]['title']
    print(str(i+1) + " : " + title)

By using the `Wikipedia` wrapper for the API we can retrieve the same information with a much more simple call.

In [ ]:
jp = wikipedia.page('Jurassic Park (novel)')

Now I have created a function in `wikisearch` which uses a FreqDist to find the 50 most common words in a page.

In [ ]:
wikisearch.get_50_most_common(wikipedia.page("Batman: No Man's Land").content)[:10]

In [ ]:
wikisearch.get_50_most_common(wikipedia.page('Catwoman').content)[:10]

In [ ]:
wikisearch.get_50_most_common(wikipedia.page('Bread').content)[:20]

In [ ]:
wikisearch.get_50_most_common(wikipedia.page('Cake').content)[:10]

In [ ]:
wikisearch.normalized_top_50(wikipedia.page("Spider-man").content)[:20]

In [ ]:
c = wikipedia.page('Trickster (comics)')
t = wikipedia.page('Catwoman')

In `wikisearch` I have created a function to make a Word2Vec model based on 2 documents, which will be used to create models based on the target page and the current page, in order to compare links to the target subject.

In [ ]:
model = make_model(c,t)

dist = model.wv.distance('also'.lower(),'Catwoman'.lower())

model.wv.most_similar(['catwoman'])

In [ ]:
model.wv.distance('catwoman', 'elseworlds')

### Main Program

The main program for running WikiGolf.

In [3]:
#get input
start = input()
target = input()
#set up initial variables
target_page = wikipedia.page(target)
closest = ("none",sys.maxsize)
print("0 : " + wikipedia.page(start).title)
# if 'path' in locals():
#     if target_page not in path:
path = [target_page]
visited = set()
visited.add(start)
title = start
exit = False
#i = 0

for i in range(100):
#while not exit:
    try:
        #test for target page
        if title.lower() == target.lower():
            exit = True
            break
        #get next page
        current = wikipedia.page(title)
        #if current not in path:
        path.append(current)
        #create model
        model = wikisearch.make_model_new(path)
        #get common words in current page
        top_20 = wikisearch.get_50_most_common(current.content)[:20]

        #test for similarity to target page
        for word,freq in top_20:
            try:
                for target_word in wikisearch.preprocess(target_page.title):
                    dist = model.wv.distance(word,target_word)
                    if closest[1] < dist:
                            closest = (title, dist)
            except KeyError:
                pass
        search_success = True
    except wikipedia.exceptions.DisambiguationError:
        search_success = False
    except wikipedia.exceptions.PageError:
        search_success = False
        
    #get next link
    title = wikisearch.check_links(model,current,target,visited)
    visited.add(title)
    if i % 10 == 0:
        print(".",end="")
    #i += 1

print("Page Found!") if exit else print("Not Found, closest page was: " + closest[0])
print(str(i) + " Hop(s), Finish at : " + title)
#print("Word: " + word + " with dist: " + str(dist))

 Batman
 Jurassic Park


0 : Batman
..........Not Found, closest page was: none
99 Hop(s), Finish at : Lincoln Park Zoo


In [ ]:
vec = make_model(wikipedia.page('Batman'),wikipedia.page('Superman'))

In [ ]:
vec.wv.distance('luthor','superman')

In [2]:
title, found = wikisearch.run_golf('Batman','Jurassic Park')
print(title)

.Jurassic Park


In [19]:
path

[<WikipediaPage 'Saint Seiya'>,
 <WikipediaPage 'Dragon Ball'>,
 <WikipediaPage 'Saint (manhua)'>,
 <WikipediaPage 'Journey to the West – Legends of the Monkey King'>,
 <WikipediaPage 'Gao Village Arc'>,
 <WikipediaPage 'Xin (comics)'>,
 <WikipediaPage 'Harris Publications'>,
 <WikipediaPage 'Guitar World'>,
 <WikipediaPage 'Play (UK magazine)'>,
 <WikipediaPage 'PlayStation'>,
 <WikipediaPage 'Scuzz'>,
 <WikipediaPage 'Starz'>,
 <WikipediaPage 'Matt's Monsters'>,
 <WikipediaPage 'Cartoon Network'>,
 <WikipediaPage 'Jim Samples'>,
 <WikipediaPage 'Disney Channel'>,
 <WikipediaPage 'Middle East'>,
 <WikipediaPage 'Sanaa'>,
 <WikipediaPage 'South Arabia'>,
 <WikipediaPage 'Upper Yafa'>,
 <WikipediaPage 'Al-Busi'>,
 <WikipediaPage 'Federation of Arab Emirates of the South'>,
 <WikipediaPage 'Kathiri'>,
 <WikipediaPage 'Wahidi Haban'>,
 <WikipediaPage 'Upper Yafa'>,
 <WikipediaPage 'Wahidi Bir Ali'>,
 <WikipediaPage 'Bi'r `Ali'>,
 <WikipediaPage 'Time zone'>,
 <WikipediaPage 'Collectivity 